In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time

def check_for_error_message(driver, error_message):
    elements = driver.find_elements(By.TAG_NAME, 'p') + driver.find_elements(By.TAG_NAME, 'li')
    for element in elements:
        if error_message in element.text:
            return True
    return False
# Initialize WebDriver
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get("https://prepaid.t-mobile.com/direct-to-account?icid=MGPR_TMO_C_CUSTSUPT_PE7I1K07DR3HZU5OA19473")

results = []

# Read phone numbers from file and remove country code
# with open('USCCphonenumber.txt', 'r') as file:
#     phone_numbers = [line.strip()[1:] if line.strip().startswith('1') else line.strip() for line in file]

try:
    for phone_number in phone_numbers:
        # Wait for the first phone number input field and enter the phone number one character at a time
        primary_phone_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "tmo-input-default-17"))
        )
        primary_phone_input.clear()
        for digit in phone_number:
            primary_phone_input.send_keys(digit)
            time.sleep(0.5)  # Mimic human typing

        # Wait a moment for possible error messages to appear
        time.sleep(2)

        # # Check if the tourist plan error message is displayed after the first input
        # if driver.find_elements(By.CSS_SELECTOR, "p.m-b-5.xs-m-b-13"):
        #     results.append((phone_number, 0))  # Tourist plan error
        #     continue  # Skip this iteration and continue with the next phone number

        # Check for the confirmation checkmark
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "i.fa.fa-check.def.active.m-r-sm"))
        )

        # Enter the phone number again in the second input field one character at a time
        confirm_phone_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "tmo-input-default-18"))
        )
        confirm_phone_input.clear()
        for digit in phone_number:
            confirm_phone_input.send_keys(digit)
            time.sleep(0.2)  # Mimic human typing

        # Click an empty space on the page
        empty_space = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.TAG_NAME, "header"))
        )
        empty_space.click()

        # Wait for possible error messages to appear after second input
        time.sleep(1)

        # Check for error messages after clicking the empty space
        # Check for all possible error messages
        if check_for_error_message(driver, "Refills and Payments are not allowed on the Tourist rate plan"):
            error_code = 0
        elif check_for_error_message(driver, "Mobile number is invalid. Please try again."):
            error_code = 1
        elif check_for_error_message(driver, "Mobile number is invalid."):
            error_code = 2
        else:
            error_code = 3  # No error found

        results.append((phone_number, error_code))

        # Refresh the page to reset state for next input
        driver.refresh()

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    driver.quit()

# Convert the results to a DataFrame
df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code'])

# Print results
print(df_results)


  Phone Number  Error Code
0   8259746686           2
1   6179910927           2
2   9105286730           0
3   8572061835           3
